In [4]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
def remove_multicollinearity(dataframe, threshold=0.9):
    """
    删除具有多重共线性的特征
    """
    corr_matrix = dataframe.corr().abs()
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]
    return dataframe.drop(columns=to_drop, errors='ignore')

def process_symbol_data(symbol, input_dir, output_dir, 
                        raw_ta_filename,
                        target_shift=-1, 
                        train_ratio=0.7, test_ratio=0.15, 
                        correlation_threshold=0.001,
                        start_index=10000):
    """
    对指定币对的数据进行处理并保存。
    """
    file_path = os.path.join(input_dir, raw_ta_filename)
    
    # 读取数据
    df = pd.read_csv(file_path)
    df = df.drop(columns=['Unnamed: 0'], errors='ignore')
    df = df.set_index('date')
    
    # 计算 return
    df['return'] = df['close'].pct_change()
    df = df.iloc[start_index:]
    nan_cols = df.columns[df.isna().any()]
    df = df.drop(columns=nan_cols)
    
    # 定义目标变量
    df['target'] = df['return'].shift(target_shift)
    if target_shift < 0:
        df = df.iloc[:target_shift, :]
    else:
        df = df.iloc[target_shift:, :]
    
    drop_cols = ['Unnamed: 0', 'date'] 
    feature_cols = [c for c in df.columns if c not in drop_cols + ['target']]
    
    X = df[feature_cols].values
    y = df['target'].values
    
    if np.isnan(X).sum() > 0 or np.isnan(y).sum() > 0:
        X = np.nan_to_num(X, nan=np.nanmean(X))
        y = np.nan_to_num(y, nan=np.nanmean(y))
    
    # 初步划分数据集
    N = len(df)
    train_size = int(N * train_ratio)
    test_size = int(N * test_ratio)
    
    train_end = train_size
    test_end = train_size + test_size
    
    X_train = X[:train_end]
    y_train = y[:train_end]
    X_train_df = pd.DataFrame(X_train, columns=feature_cols)
    y_train_series = pd.Series(y_train, name='target')
    
    df_temp = pd.concat([X_train_df, y_train_series], axis=1)
    df_filtered = remove_multicollinearity(df_temp, threshold=0.9)
    
    if 'target' not in df_filtered.columns:
        df_filtered['target'] = y_train_series
    correlations = df_filtered.corr(method='pearson')['target'].drop('target')
    selected_features = correlations[correlations.abs() > correlation_threshold].index.tolist()
    df_filtered = df_filtered[selected_features + ['target']]
    
    all_features_df = pd.DataFrame(X, columns=feature_cols)
    all_df = pd.concat([all_features_df, pd.Series(y, name='target')], axis=1)
    all_df = all_df[selected_features + ['target']]
    
    train_size = int(N * train_ratio)
    test_size = int(N * test_ratio)
    
    train_end = train_size
    test_end = train_size + test_size
    
    X_all = all_df.drop('target', axis=1).values
    y_all = all_df['target'].values
    
    X_train = X_all[:train_end]
    y_train = y_all[:train_end]

    X_test = X_all[train_end:test_end]
    y_test = y_all[train_end:test_end]

    X_val = X_all[test_end:]
    y_val = y_all[test_end:]
    
    X_train_df = pd.DataFrame(X_train, columns=selected_features)
    X_val_df = pd.DataFrame(X_val, columns=selected_features)
    X_test_df = pd.DataFrame(X_test, columns=selected_features)
    
    X_train_df['target'] = y_train
    X_val_df['target'] = y_val
    X_test_df['target'] = y_test
    
    # 创建输出目录
    symbol_dir = os.path.join(output_dir, symbol)
    os.makedirs(symbol_dir, exist_ok=True)
    
    train_file_path = os.path.join(symbol_dir, f"Xy_train_{symbol}.csv")
    val_file_path = os.path.join(symbol_dir, f"Xy_val_{symbol}.csv")
    test_file_path = os.path.join(symbol_dir, f"Xy_test_{symbol}.csv")

    X_train_df.to_csv(train_file_path, index=False)
    X_val_df.to_csv(val_file_path, index=False)
    X_test_df.to_csv(test_file_path, index=False)
    
    print(f"{symbol}: Processed files saved at {symbol_dir}")

def process_all_symbols(input_dir='.', output_dir='.', **kwargs):
    """
    读取输入目录下的所有符合 *_1m_ta.csv 格式的文件，并进行处理。
    """
    # 获取所有符合 *_1m_ta.csv 的文件
    ta_files = [f for f in os.listdir(input_dir) if f.endswith('_1m_ta.csv')]
    
    # 提取交易对名称并处理
    for ta_file in tqdm(ta_files, desc="Processing all symbols"):
        symbol = ta_file.split('_1m_ta.csv')[0]
        process_symbol_data(symbol, input_dir=input_dir, output_dir=output_dir, raw_ta_filename=ta_file, **kwargs)

In [ ]:
process_all_symbols(input_dir='data', output_dir='processed', start_index=10000, correlation_threshold=0.001)


Processing all symbols:   0%|          | 0/106 [00:00<?, ?it/s]

Processing all symbols:   1%|          | 1/106 [00:27<47:52, 27.36s/it]

1000SATSUSDT: Processed files saved at processed\1000SATSUSDT


Processing all symbols:   2%|▏         | 2/106 [00:55<47:58, 27.68s/it]

AAVEUSDT: Processed files saved at processed\AAVEUSDT


...

TIAUSDT: Processed files saved at processed\TIAUSDT


Processing all symbols:  87%|████████▋ | 92/106 [39:23<04:21, 18.71s/it]

TONUSDT: Processed files saved at processed\TONUSDT


Processing all symbols:  88%|████████▊ | 93/106 [39:48<04:27, 20.54s/it]

TRBUSDT: Processed files saved at processed\TRBUSDT
